### Install dependencies

In [ ]:
%%bash
git clone https://github.com/KaiyangZhou/Dassl.pytorch.git
cd Dassl.pytorch/
# Install dependencies
pip install -r requirements.txt
# Install this library (no need to re-build if the source code is modified)
python setup.py develop
cd ..
# Install dependencies of MaPLe
pip install -r requirements.txt

### Download dataset (OxfordPets)

In [ ]:
%%bash
cd ../../datasets/data
mkdir oxford_pets
cd oxford_pets
wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
gdown 1501r8Ber4nNKvmlFVQZ8SeUHTcdTTEqs
tar -zxvf images.tar.gz
tar -zxvf annotations.tar.gz

### Run MaPLe

In [ ]:
%%bash
# seed=1
# trains and evaluates on base classes
bash scripts/maple/base2new_train_maple.sh oxford_pets 1 ../../datasets/data/
# evaluates on novel classes
bash scripts/maple/base2new_test_maple.sh oxford_pets 1 ../../datasets/data/

In [ ]:
%%bash
# seed=2
# trains and evaluates on base classes
bash scripts/maple/base2new_train_maple.sh oxford_pets 2 ../../datasets/data/
# evaluates on novel classes
bash scripts/maple/base2new_test_maple.sh oxford_pets 2 ../../datasets/data/

In [ ]:
%%bash
# seed=3
# trains and evaluates on base classes
bash scripts/maple/base2new_train_maple.sh oxford_pets 3 ../../datasets/data/
# evaluates on novel classes
bash scripts/maple/base2new_test_maple.sh oxford_pets 3 ../../datasets/data/